# BeaverTails-Evaluation dataset [beaver_tails_dataset.json]
- Load via Hugging Face datasets: https://huggingface.co/datasets/PKU-Alignment/BeaverTails-Evaluation
- The official split contains 700 samples across 14 categories.
- Save locally without any sampling to ./my_dataset/beaver_tails_dataset.json and also use save_to_disk for reproducibility.

In [ ]:
from pathlib import Path
from datasets import load_dataset

# Load the dataset
ds = load_dataset("PKU-Alignment/BeaverTails-Evaluation")

# Save the full dataset to a local directory (relative path, obfuscated for GitHub)
DATA_DIR = Path("./my_dataset")
DATA_DIR.mkdir(parents=True, exist_ok=True)
local_directory = DATA_DIR / "beaver_tails_dataset"
ds.save_to_disk(str(local_directory))

print(f"Dataset saved to {local_directory}")

In [ ]:
from pathlib import Path
from datasets import load_dataset

# Load the dataset
ds = load_dataset('PKU-Alignment/BeaverTails-Evaluation')

# Save split to JSON (relative path for GitHub)
DATA_DIR = Path("./my_dataset")
DATA_DIR.mkdir(parents=True, exist_ok=True)
json_file_path = DATA_DIR / 'beaver_tails_dataset.json'

# Convert the dataset to JSON and save it
ds['test'].to_json(str(json_file_path))

print(f"Dataset saved as JSON to {json_file_path}")

# HarmfulQA dataset [harmfulqa.jsonl]
- Load declare-lab/HarmfulQA: https://huggingface.co/datasets/declare-lab/HarmfulQA
- 1,960 harmful questions across 10 topics (with subtopics).
- Randomly select 700 samples balanced across topics (70 per topic; subtopics are not balanced).
- Extract question/topic/subtopic and save to ./my_dataset/harmfulqa.jsonl.
- Use random.seed(42) for reproducibility.

In [ ]:
import random
import json
from pathlib import Path
from datasets import load_dataset

# Load the HarmfulQA dataset
ds = load_dataset("declare-lab/HarmfulQA")

# Set the random seed for reproducibility
random.seed(42)

def balanced_random_sample(dataset, output_file_path, num_samples=700):
    """
    Sample an approximately balanced subset by 'topic' and write it as an intermediate JSON array.
    The final JSONL with selected fields will be generated in the next step.
    """
    topics = dataset['train'].unique('topic')
    num_topics = len(topics)
    if num_topics == 0:
        raise ValueError("No topics found in HarmfulQA train split.")

    samples_per_topic = max(1, num_samples // num_topics)
    sampled_data = []

    for topic in topics:
        # Filter data by topic, convert to a list of dicts
        topic_data = list(dataset['train'].filter(lambda x: x['topic'] == topic))
        k = min(samples_per_topic, len(topic_data))
        if k > 0:
            sampled_data.extend(random.sample(topic_data, k))

    # Write the sampled data to JSON
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(sampled_data, output_file, ensure_ascii=False, indent=2)

    return sampled_data

# Example usage (relative, obfuscated path):
DATA_DIR = Path("./my_dataset")
DATA_DIR.mkdir(parents=True, exist_ok=True)
output_file_path = DATA_DIR / 'harmfulqa_balanced_sample.json'
sampled_data = balanced_random_sample(ds, str(output_file_path))

print(f"Balanced sample saved to {output_file_path}")

In [ ]:
import json
from pathlib import Path

def convert_to_jsonl(input_json_path, output_jsonl_path):
    """
    Convert the intermediate JSON array from balanced_random_sample into JSONL.
    Each line contains: {"topic": "...", "subtopic": "...", "question": "..."}
    """
    with open(input_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    with open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
        for sample in data:
            topic = sample.get('topic', 'unknown_topic')
            subtopic = sample.get('subtopic', 'unknown_subtopic')
            question = sample.get('question', 'unknown_question')
            json_line = json.dumps({
                "topic": topic,
                "subtopic": subtopic,
                "question": question
            }, ensure_ascii=False)
            outfile.write(json_line + '\n')

    print(f"JSONL file saved to {output_jsonl_path}")
    return output_jsonl_path

In [ ]:
# Convert to JSONL
from pathlib import Path

DATA_DIR = Path("./my_dataset")
output_json_path = DATA_DIR / 'harmfulqa_balanced_sample.json'
output_jsonl_path = DATA_DIR / 'harmfulqa.jsonl'
convert_to_jsonl(str(output_json_path), str(output_jsonl_path))

# TruthfulQA dataset [TruthfulQA.jsonl]
- Download CSV from the original repository (790 samples):
  https://github.com/sylinrl/TruthfulQA/blob/main/TruthfulQA.csv
- Convert to JSONL for easier downstream processing.

In [ ]:
import csv
import json
from pathlib import Path

# Input/Output paths (relative, for GitHub)
DATA_DIR = Path("./my_dataset")
DATA_DIR.mkdir(parents=True, exist_ok=True)
csv_file_path = DATA_DIR / 'TruthfulQA.csv'
jsonl_file_path = DATA_DIR / 'TruthfulQA.jsonl'

with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
        for row in reader:
            jsonl_file.write(json.dumps(row, ensure_ascii=False) + '\n')

# DialogSum dataset [dialogsum_test.jsonl]
- Use the test split provided by the authors (500 samples):
  https://github.com/cylnlp/dialogsum/blob/main/DialogSum_Data/dialogsum.test.jsonl
- Place it under: ./my_dataset/dialogsum_test.jsonl

# EmpatheticDialogues (Empathy) [empathy_test.jsonl]
- Original repository: https://github.com/facebookresearch/EmpatheticDialogues
- Download data: wget https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz
- Use the test split; select 16 negative emotion categories; deduplicate prompts; filter out too-short texts; sample up to 40 per category (seed=42).
- Save the final 640 examples to ./my_dataset/empathy_test.jsonl

In [ ]:
import pandas as pd
import json
from pathlib import Path

# Read the CSV (handle quoting/escaping issues)
ROOT = Path(".")
DATA_DIR = ROOT / "my_dataset"
DATA_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(
    ROOT / 'empatheticdialogues' / 'test.csv',
    quotechar='"',
    escapechar='\\',
    on_bad_lines='skip'
)

# Target emotion categories (16)
emotions = ["angry", "confident", "embarrassed", "proud", "sad", "lonely", "terrified", "disgusted", "devastated", "annoyed", "ashamed", "anxious", "jealous", "disappointed", "afraid", "guilty"]

# Filter rows and drop duplicate prompts
filtered_df = df[df['context'].isin(emotions)].drop_duplicates(subset=['prompt'])

# Emit category/prompt pairs as JSONL
out_jsonl = DATA_DIR / 'empathy.jsonl'
with open(out_jsonl, 'w', encoding='utf-8') as f:
    for _, row in filtered_df.iterrows():
        obj = {'category': row['context'], 'prompt': row['prompt']}
        f.write(json.dumps(obj, ensure_ascii=False) + '\n')

print(f"Wrote {len(filtered_df)} lines to {out_jsonl}")

In [ ]:
import json
import random
from pathlib import Path

# Reproducibility
random.seed(42)

emotions = ["angry", "confident", "embarrassed", "proud", "sad", "lonely", "terrified",
            "disgusted", "devastated", "annoyed", "ashamed", "anxious", "jealous",
            "disappointed", "afraid", "guilty"]

DATA_DIR = Path("./my_dataset")
input_path = DATA_DIR / 'empathy.jsonl'
with open(input_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Clean and filter
cleaned = [
    {"category": d.get("category"), "prompt": d.get("prompt", "").replace("_comma_", ",")}
    for d in data
    if d.get("category") in emotions and len(d.get("prompt", "")) >= 40
]

# Group and sample
by_cat = {emo: [] for emo in emotions}
for item in cleaned:
    by_cat[item["category"]].append(item)
final_data = [
    s
    for emo in emotions
    for s in random.sample(by_cat[emo], min(40, len(by_cat[emo])))
]

# Write output
output_path = DATA_DIR / 'empathy_test.jsonl'
with open(output_path, 'w', encoding='utf-8') as f:
    for item in final_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"Data processing complete, written to {output_path}, total samples: {len(final_data)}")

## Summary

| Dataset | Classes | Samples | Path |
| --- | --- | --- | --- |
| empathy_test.jsonl | 16 | 640 | ./my_dataset/empathy_test.jsonl |
| dialogsum_test.jsonl | - | 500 | ./my_dataset/dialogsum_test.jsonl |
| TruthfulQA.jsonl | - | 790 | ./my_dataset/TruthfulQA.jsonl |
| beaver_tails_dataset.json | 14 | 700 | ./my_dataset/beaver_tails_dataset.json |
| harmfulqa.jsonl | 10 | 700 | ./my_dataset/harmfulqa.jsonl |

# Shuffle dataset order

In [ ]:
import random
from pathlib import Path

def shuffle_jsonl(input_path, output_path, seed=42):
    """Shuffle lines of a JSONL file deterministically by seed."""
    random.seed(seed)
    input_path = Path(input_path)
    output_path = Path(output_path)
    lines = input_path.read_text(encoding='utf-8').splitlines()
    random.shuffle(lines)
    output_path.write_text(('\n'.join(lines) + ('\n' if lines else '')), encoding='utf-8')
    print(f"Shuffled dataset written to {output_path}, total {len(lines)} records")

In [ ]:
# Example: shuffle empathy_test.jsonl in-place
shuffle_jsonl(
    input_path="./my_dataset/empathy_test.jsonl",
    output_path="./my_dataset/empathy_test.jsonl",
    seed=42
)

In [ ]:
# Example: shuffle harmfulqa.jsonl in-place
shuffle_jsonl(
    input_path="./my_dataset/harmfulqa.jsonl",
    output_path="./my_dataset/harmfulqa.jsonl",
    seed=42
)

In [ ]:
from datasets import load_dataset

# Load the BeaverTails dataset and print the split info
dataset = load_dataset('PKU-Alignment/BeaverTails-Evaluation')
print(dataset)

In [ ]:
# Snippet for papers/docs

import json
import random
from collections import defaultdict

# Set random seed for reproducibility
random.seed(42)
emotions = ["angry", "confident", "embarrassed", "proud", "sad", "lonely", "terrified",
            "disgusted", "devastated", "annoyed", "ashamed", "anxious", "jealous",
            "disappointed", "afraid", "guilty"]

# Read the original jsonl file
input_path = ""  # your path here
with open(input_path, "r", encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Preprocess: replace _comma_ and filter out short texts
cleaned_data = []
for item in data:
    category = item["category"]
    prompt = item["prompt"].replace("_comma_", ",")
    if category in emotions and len(prompt) >= 40:
        cleaned_data.append({"category": category, "prompt": prompt})

# Sample up to 40 items per emotion category
samples_per_emotion = defaultdict(list)
for item in cleaned_data:
    samples_per_emotion[item["category"]].append(item)

# Randomly sample 40 items per category
final_data = []
for emotion in emotions:
    samples = samples_per_emotion[emotion]
    if len(samples) >= 40:
        final_data.extend(random.sample(samples, 40))
    else:
        print(f"Not enough samples for emotion '{emotion}': only {len(samples)}; keeping all.")

# Shuffle the order of the data randomly
random.shuffle(final_data)

# Write to new file
output_path = ""  # your path here
with open(output_path, "w", encoding='utf-8') as f:
    for item in final_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"Data processing complete. Written to {output_path}. Total samples: {len(final_data)}")

In [ ]:
# Standalone snippet for HarmfulQA balanced sampling

import random
import json
from datasets import load_dataset

# Load the HarmfulQA dataset
ds = load_dataset("declare-lab/HarmfulQA")

# Set the random seed for reproducibility
random.seed(42)

def balanced_random_sample(dataset, output_file_path, num_samples=700):
    # List unique topics
    topics = dataset['train'].unique('topic')
    num_topics = len(topics)
    if num_topics == 0:
        raise ValueError("No topics found in HarmfulQA train split.")

    # Calculate target per topic
    samples_per_topic = max(1, num_samples // num_topics)

    sampled_data = []
    for topic in topics:
        topic_data = list(dataset['train'].filter(lambda x: x['topic'] == topic))
        k = min(samples_per_topic, len(topic_data))
        if k > 0:
            sampled_data.extend(random.sample(topic_data, k))

    # Write the sampled data to the output file in JSON format
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(sampled_data, output_file, ensure_ascii=False, indent=2)

    return sampled_data

# Example usage:
output_file_path = './my_dataset/harmfulqa_balanced_sample.json'  # your path here
sampled_data = balanced_random_sample(ds, output_file_path)

print(f"Balanced sample saved to {output_file_path}")